### 导包和变量设置

In [1]:
# 引入PyPDFDirectoryLoader，可以从文件夹中一次性加载所有pdf文件
# 然后使用RecursiveCharacterTextSplitter对解析出来的文档进行切分，主要根据分隔符，chunk_size以及overlap等

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers.bm25 import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import os
import gc
import pickle

DOCS_DIR = '/root/autodl-tmp/dataset/rag/A_document'
EMB_MODEL = '/root/autodl-tmp/models/bge-large-zh-v1_5'
RERANK_MODEL = "/root/autodl-tmp/models/bge-reranker-large"
PERSIST_DIR = '/root/autodl-tmp/vectorDatabase/faiss_llmsherpa'
QUERY_DIR = '/root/autodl-tmp/dataset/rag/A_question.csv'
SUB_DIR = '/root/autodl-tmp/dataset/rag/submit.csv'
# query = pd.read_csv(QUERY_DIR)

path = "/root/autodl-tmp/dataset/rag/query.pkl"

with open(path, "rb") as f:
    query = pickle.load(f)

sub = pd.read_csv("/root/autodl-tmp/dataset/rag/submit_example.csv")
display(query.head(3))
display(sub.head(3))

/root/miniconda3/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,ques_id,question,question_fyde
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,根据2022年年度报告，中国联通在向数字科技领军企业转变的过程中，实现了以下维度的转型升级：...
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,2022年中国联通产业互联网收入同比增长速度为13.8%。
2,3,根据2022年度报告，中国联通的企业定位是什么？,根据2022年度报告，中国联通的企业定位是：\n\n1. 成为数字创新服务领导者\n2. 推...


,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."
2,3,根据2022年度报告，中国联通的企业定位是什么？,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."


In [2]:
# questions = list(query['question_fyde'].values)

In [3]:
# # 针对query使用hyde方法进行扩充
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate

# # from yvan_llm_practice.glm4.glm4LLM_VLLM import ChatGLM4_LLM
# # import os
# # import sys
# # # 获取当前工作目录
# # current_dir = os.getcwd()
# # # 获取上一级目录
# # parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# # # 将上一级目录添加到 sys.path
# # sys.path.insert(0, parent_dir)

# from glm4.glm4LLM_VLLM import ChatGLM4_LLM

# # system = """You are an expert about a set of software for building LLM-powered applications called LangChain, LangGraph, LangServe, and LangSmith.

# # LangChain is a Python framework that provides a large set of integrations that can easily be composed to build LLM applications.
# # LangGraph is a Python package built on top of LangChain that makes it easy to build stateful, multi-actor LLM applications.
# # LangServe is a Python package built on top of LangChain that makes it easy to deploy a LangChain application as a REST API.
# # LangSmith is a platform that makes it easy to trace and test LLM applications.

# # Answer the user question as best you can. Answer as though you were writing a tutorial that addressed the user question."""

# system = """你是一名中国联通的专家，精通公司内部的各项业务和技术。你具备以下背景知识：

# 技术前沿：深入了解5G、物联网、大数据和人工智能在通信行业的应用和发展方向。
# 数字化转型：帮助企业和政府客户实现数字化转型，通过智能化解决方案提升效率和竞争力。
# 全球视野：熟悉中国联通在国际市场的布局和合作策略，推动全球通信网络的互联互通。
# 创新驱动：关注技术创新，支持公司在云计算、区块链等新兴领域的探索和应用。
# 用户导向：以用户为中心，致力于提升服务质量和用户体验，满足多样化的客户需求。
# 社会责任：积极参与公益事业，推动教育和环保项目，履行企业社会责任。

# 尽可能好地回答用户问题。回答时要像在写一个教程，以解决用户的问题。
# 一定要注意，直接回答问题，不需要多余的任何冗余"""


# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system),
#         ("human", "{question}"),
#     ]
# )
# # llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# llm = ChatGLM4_LLM(api_base_url="http://localhost:8000/v1")
# qa_no_context = prompt | llm | StrOutputParser()

In [4]:
# question = "2019年底和2020年底，联通宽带端口总数分别为多少？"

# answer = qa_no_context.invoke(
#     {
#         "question": question
#     }
# )
# print(answer)



In [5]:
# def data_process(question):
#     answer = qa_no_context.invoke(
#         {
#             "question": question
#         }
#     )
#     return answer

# query['question_fyde'] = query.apply(data_process, axis=1)

In [6]:
# import pickle 

# path = "/root/autodl-tmp/dataset/rag/query.pkl"
# with open(path, "wb") as f:
#     pickle.dump(result, f)

# result["question_fyde"] = result["question_fyde"].apply(lambda x: x.strip())


# with open(path, "rb") as f:
#     result = pickle.load(f)

### PDF文档解析和切分

In [7]:
# # # 进行数据加载
# # loader = PyPDFDirectoryLoader(DOCS_DIR)

# # 使用 PyPDFDirectoryLoader 加载所有 PDF 文件
# pdf_loader = PyPDFDirectoryLoader(DOCS_DIR)
# documents = pdf_loader.load()

# # 使用 LLMSherpaFileLoader 加载文档
# sherpa_loader = LLMSherpaFileLoader(
#     new_indent_parser=True,
#     apply_ocr=False,
#     strategy="text",
#     # llmsherpa_api_url="http://127.0.0.1:5001/api/parseDocument?renderFormat=all&useNewIndentParser=true&applyOcr=yes"
#     llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all",
# )

# loaded_documents = [sherpa_loader.load(doc) for doc in documents]

# from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader

# loader = LLMSherpaFileLoader(
#     file_path="/root/autodl-tmp/dataset/rag/A_small/AF01.pdf",
#     new_indent_parser=True,
#     apply_ocr=False,
#     strategy="text",
#     llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all",
# )

# docs = loader.load_and_split(
#     RecursiveCharacterTextSplitter(        
#         chunk_size=200,             
#         chunk_overlap=0,
#         separators = ["。", "！", "？"],
#         keep_separator='end',
#     ),
# )
# # 打印文档数量
# print(len(docs))
# # print(docs[0].page_content)

# # 打印所有第一页的数据出来看下，切分效果如何
# for i, item in enumerate(docs):
#     print(f"the {i} doc's content i: {item.page_content}")

In [8]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader


# 指定 PDF 文件夹路径
pdf_directory = DOCS_DIR

# 获取文件夹中所有 PDF 文件的路径
pdf_files = [os.path.join(pdf_directory, f) for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# 存储所有切分后的文档块
all_split_documents = []

# 对每个 PDF 文件进行处理
for pdf_file in pdf_files:
    # 使用 LLMSherpaFileLoader 加载文档
    loader = LLMSherpaFileLoader(
        file_path=pdf_file,
        new_indent_parser=True,
        apply_ocr=False,
        strategy="text",
        llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all"
    )
    
    # 使用 RecursiveCharacterTextSplitter 切分文档
    docs_small = loader.load_and_split(
        RecursiveCharacterTextSplitter(
            chunk_size=100,
            chunk_overlap=30,
            separators=["。", "！", "？"],
            # separators=["。"],
            keep_separator='end',
        )
    )

    # docs_big = loader.load_and_split(
    #     RecursiveCharacterTextSplitter(
    #         chunk_size=200,
    #         chunk_overlap=0,
    #         separators=["。", "！", "？"],
    #         # separators=["。"],
    #         keep_separator='end',
    #     )
    # )
    
    # 将切分后的文档块添加到列表中
    all_split_documents.extend(docs_small)

# # 输出所有切分后的文档块
# print(all_split_documents)

In [10]:
len(all_split_documents)

14067

In [11]:
for i in range(30):
    print("##########" * 5 + f"{i+1}" + "##########" * 5)
    print(all_split_documents[i].page_content)

##################################################1##################################################
中国联通人工智能创新发展论坛在上海成功举办
发布时间：2024 年
7
月 20 日 2024 年
7
月 19 日，在中国联通合作伙伴大会期间，成功举办了人工智能创新发展论坛。
##################################################2##################################################
 上海市经信委副主任张宏韬、中国联通总经理简勤、GSMA 中华区总裁斯寒出席论坛并致辞； 中国工程院院士谭建荣，加拿大工程院院士、欧洲科学院院士、香港科技大学教授郭嵩，联 通数字科技有限公司总裁、中国联通人工智能创新中心主任朱常波，中国联通人工智能科学 家兼人工智能技术总师廉士国，中国联通数字化部副总经理娄瑜发表主旨演讲。
##################################################3##################################################

上海市经信委副主任张宏韬在致辞中表示，中国联通作为中央企业，深入贯彻落实国家 “人工智能+”专项行动，在人工智能领域取得了令人瞩目的成就，中国联通人工智能创新中 心充分利用中国联通在网、算、云、数、智、端、业的融合优势，推动了人工智能创新应用 规模化发展，展现了央企在新时代的责任与担当。
##################################################4##################################################
上海市人民政府也高度重视人工智能的发 展，致力于打造开放的创新平台，吸引全球人工智能企业和人才汇聚，共同推动技术交流和 国际合作。
##################################################5##################################################
中国联通总经理简勤在致辞中表示，元景 2.0 不仅

In [12]:
for doc in all_split_documents:
    # doc.page_content = doc.page_content.replace(" ", "").replace("\n", "")

    # 没有替换\n，因为有些表格，不应该删除换行
    doc.page_content = doc.page_content.replace(" ", "")

In [13]:
for i in range(50):
    print("##########" * 5 + f"{i+1}" + "##########" * 5)
    print(all_split_documents[i].page_content)

##################################################1##################################################
中国联通人工智能创新发展论坛在上海成功举办
发布时间：2024年
7
月20日2024年
7
月19日，在中国联通合作伙伴大会期间，成功举办了人工智能创新发展论坛。
##################################################2##################################################
上海市经信委副主任张宏韬、中国联通总经理简勤、GSMA中华区总裁斯寒出席论坛并致辞；中国工程院院士谭建荣，加拿大工程院院士、欧洲科学院院士、香港科技大学教授郭嵩，联通数字科技有限公司总裁、中国联通人工智能创新中心主任朱常波，中国联通人工智能科学家兼人工智能技术总师廉士国，中国联通数字化部副总经理娄瑜发表主旨演讲。
##################################################3##################################################

上海市经信委副主任张宏韬在致辞中表示，中国联通作为中央企业，深入贯彻落实国家“人工智能+”专项行动，在人工智能领域取得了令人瞩目的成就，中国联通人工智能创新中心充分利用中国联通在网、算、云、数、智、端、业的融合优势，推动了人工智能创新应用规模化发展，展现了央企在新时代的责任与担当。
##################################################4##################################################
上海市人民政府也高度重视人工智能的发展，致力于打造开放的创新平台，吸引全球人工智能企业和人才汇聚，共同推动技术交流和国际合作。
##################################################5##################################################
中国联通总经理简勤在致辞中表示，元景2.0不仅是中国联通人工智能技术的升级，更是对人

In [ ]:
# # 使用MinerU进行文档提取(简直无语了，巨慢，8页纸居然用了2min5s左右)
# import os

# from loguru import logger

# from magic_pdf.data.data_reader_writer import FileBasedDataWriter
# from magic_pdf.pipe.UNIPipe import UNIPipe



# try:
#     # current_script_dir = os.path.dirname(os.path.abspath(__file__))
#     # demo_name = 'demo1'
#     # pdf_path = os.path.join(current_script_dir, f'{demo_name}.pdf')

#     current_script_dir = "/root/autodl-tmp/dataset/rag/A_small"
#     demo_name = "AF01"
#     pdf_path = os.path.join(current_script_dir, f'{demo_name}.pdf')
#     pdf_bytes = open(pdf_path, 'rb').read()
#     jso_useful_key = {'_pdf_type': '', 'model_list': []}
#     local_image_dir = os.path.join(current_script_dir, 'images')
#     image_dir = str(os.path.basename(local_image_dir))
#     image_writer = FileBasedDataWriter(local_image_dir)
#     pipe = UNIPipe(pdf_bytes, jso_useful_key, image_writer)
#     # pipe.pipe_classify()
#     # pipe.pipe_analyze()
#     pipe.pipe_parse()
#     md_content = pipe.pipe_mk_markdown(image_dir, drop_mode='none')
#     with open(f'{demo_name}.md', 'w', encoding='utf-8') as f:
#         f.write(md_content)
# except Exception as e:
#     logger.exception(e)


In [ ]:
# # 使用llmsherpa
# # from llmsherpa.readers import LayoutPDFReader

# # llmsherpa_api_url = "http://127.0.0.1:5001//api/parseDocument?renderFormat=all"
# # pdf_url = "/root/autodl-tmp/dataset/rag/A_small/AF01.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
# # pdf_reader = LayoutPDFReader(llmsherpa_api_url)
# # doc = pdf_reader.read_pdf(pdf_url)

# from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader

# loader = LLMSherpaFileLoader(
#     file_path="/root/autodl-tmp/dataset/rag/A_small/AF01.pdf",
#     new_indent_parser=True,
#     apply_ocr=False,
#     strategy="html",
#     # llmsherpa_api_url="http://127.0.0.1:5001/api/parseDocument?renderFormat=all&useNewIndentParser=true&applyOcr=yes"
#     llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all",
# )

# docs = loader.load()




### 文本块向量化（比赛限定使用bge-large-zh-v1.5模型）

In [14]:
embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL, show_progress=True)
vectordb = FAISS.from_documents(   
    documents=all_split_documents,
    embedding=embeddings,
)

vectordb.save_local(PERSIST_DIR)

/tmp/ipykernel_17234/2932526079.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL, show_progress=True)
Batches: 100%|██████████| 440/440 [00:56<00:00,  7.72it/s]


### 混合检索器

#### bm25 
- k1 较高的 k1 值意味着词频对评分的影响更大。
- b  当 b=1 时，文档长度的影响最大；当b = 0 时，文档长度不影响评分。
- langchain 默认切分英文split()，中文需要jieba分词

In [15]:
import jieba
dense_retriever = vectordb.as_retriever(search_kwargs={"k": 5})
bm25_retriever = BM25Retriever.from_documents(
    all_split_documents, 
    k=5, 
    bm25_params={"k1": 1.5, "b": 0.75}, 
    preprocess_func=jieba.lcut
)
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, dense_retriever], weights=[0.5, 0.5])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.685 seconds.
Prefix dict has been built successfully.


### 文本召回和重排

In [ ]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import CrossEncoderReranker
# from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# def rerank(questions, retriever, top_n=5, cut_len=384):
#     rerank_model = HuggingFaceCrossEncoder(model_name=RERANK_MODEL)
#     compressor = CrossEncoderReranker(model=rerank_model, top_n=top_n)
#     compression_retriever = ContextualCompressionRetriever(
#         base_compressor=compressor, base_retriever=retriever
#     )
#     rerank_answers = []
#     for question in tqdm(questions):
#         relevant_docs = compression_retriever.invoke(question)
#         answer=''
#         for rd in relevant_docs:
#             answer += rd.page_content
#         rerank_answers.append(answer[:cut_len])
#     return rerank_answers

# questions = list(query['question'].values)
# rerank_answers = rerank(questions, ensemble_retriever)
# print(rerank_answers[0])


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers import ContextualCompressionRetriever
from tqdm import tqdm

from datasketch import MinHash, MinHashLSH

def get_minhash(doc, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for word in doc.page_content.split():
        m.update(word.encode('utf8'))
    return m

def deduplicate_documents_minhash(documents, threshold=0.8):
    lsh = MinHashLSH(threshold=threshold, num_perm=128)
    unique_docs = []
    minhashes = []

    for i, doc in enumerate(documents):
        m = get_minhash(doc)
        minhashes.append(m)
        if not lsh.query(m):
            lsh.insert(i, m)
            unique_docs.append(doc)

    return unique_docs

def retrieve_documents(query, retriever):
    docs = retriever.invoke(query)
    # print(f"the docs is: {docs}")
    return docs

def rerank_documents(query, documents, rerank_model, top_n=5):
    compressor = CrossEncoderReranker(model=rerank_model, top_n=top_n)
    relevant_docs = compressor.compress_documents(documents, query)
    return relevant_docs


def rerank(questions, questions_fyde, retriever, rerank_model_name, top_n=5, cut_len=384):
    rerank_model = HuggingFaceCrossEncoder(model_name=rerank_model_name)
    rerank_answers = []

    for question, question_fyde in tqdm(zip(questions, questions_fyde)):
        # 单次调用召回
        docs_query = retrieve_documents(question, retriever)
        docs_fyde = retrieve_documents(question_fyde, retriever)

        # 合并文档
        all_docs = docs_query + docs_fyde

        # 去重文档
        unique_docs = deduplicate_documents_minhash(all_docs)

        # 重新排序
        reranked_docs = rerank_documents(question, unique_docs, rerank_model, top_n)

        # 提取内容
        answer = '\n'.join(doc.page_content for doc in reranked_docs)
        rerank_answers.append(answer[:cut_len])

    return rerank_answers

# 使用示例
questions = list(query['question'].values)
questions_fyde = list(query["question_fyde"].values)

rerank_answers = rerank(questions, questions_fyde, ensemble_retriever, RERANK_MODEL)
print(rerank_answers[0])

Batches: 100%|██████████| 1/1 [00:00<00:00, 84.42it/s]
100it [00:56,  1.76it/s]



我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以5G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。具体而言，中国联通将通过通信科技（CT）、信息科技（IT）、数字科技（DT）和运营科技（OT）等领域的融合创新，推动企业从传统的通信服务


### 提交

In [21]:
def emb(answers, emb_batch_size = 4):
    model = SentenceTransformer(EMB_MODEL, trust_remote_code=True).half()
    all_sentence_embeddings = []
    for i in tqdm(range(0, len(answers), emb_batch_size), desc="embedding sentences"):
        batch_sentences = answers[i:i+emb_batch_size]
        sentence_embeddings = model.encode(batch_sentences, normalize_embeddings=True)
        all_sentence_embeddings.append(sentence_embeddings)
    all_sentence_embeddings = np.concatenate(all_sentence_embeddings, axis=0)
    print('emb_model max_seq_length: ', model.max_seq_length)
    print('emb_model embeddings_shape: ', all_sentence_embeddings.shape[-1])
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return all_sentence_embeddings

all_sentence_embeddings = emb(rerank_answers)
sub['answer'] = rerank_answers
sub['embedding']= [','.join([str(a) for a in all_sentence_embeddings[i]]) for i in range(len(all_sentence_embeddings))]
sub.to_csv(SUB_DIR, index=None)
sub.head()

embedding sentences: 100%|██████████| 25/25 [00:00<00:00, 34.11it/s]


emb_model max_seq_length:  512
emb_model embeddings_shape:  1024


,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,\n我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向...,"-0.01968,-0.007393,-0.02017,-0.000923,0.01643,..."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,公司产业互联网继续按下快进键，2022年收入首破700亿大关，同比增长达到29%，实现规模、...,"-0.02527,-0.00987,-0.02826,-0.002289,0.02846,0..."
2,3,根据2022年度报告，中国联通的企业定位是什么？,\n公司基本情况\n1公司简介3公司主要会计数据和财务指标\n||公司股票简况\n|---|...,"-0.04153,-0.01656,-0.04892,0.014305,0.03244,0...."
3,4,2022年联通在“大联接”和“大数据”业务上取得了什么成果？,“大联接”方面，中国联通抢抓“双千兆”“物超人”发展机遇，坚持量质构效协同发展。“大联接”方...,"-0.02792,-0.01643,-0.02402,0.02113,0.01569,0.0..."
4,5,2022年上半年，联通在精品网络建设上有什么成果？,公司固网宽带业务延续了去年高速增长的良好态势，上半年实现宽带接入收入230亿元，同比提升适度...,"-0.00792,-0.02966,-0.02548,-0.00754,0.00131,0...."


In [23]:
for i in range(20):
    print("##########"* 5, f"{i+1}", "##########" * 5)
    print(rerank_answers[i])


################################################## 1 ##################################################

我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以5G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。具体而言，中国联通将通过通信科技（CT）、信息科技（IT）、数字科技（DT）和运营科技（OT）等领域的融合创新，推动企业从传统的通信服务
################################################## 2 ##################################################
公司产业互联网继续按下快进键，2022年收入首破700亿大关，同比增长达到29%，实现规模、增速双提升。物联网实现收入86亿元，同比增长达到42%，非连接收入增长63%，增速大幅领跑行业。大计算积厚成势，“联通云”继续翻倍增长，2022年实现收入361亿元，同比增速达到121%。2019年上半年，创新业务成为稳定公司收入的主要驱动力。产业互联网业务收入同比增长43%，达到人民币167亿元，占整体主营业务收入比例提高至13%。今年以来，联通云收入同比增长超过140%，全年收入将突破300亿元。联通云加速发展，实现收入3人民币268.7亿元，同比提升142.0%；IDC实现收入人民币186.1亿元，同比提升12.9%。
################################################## 3 ##########################################

In [ ]:
from langchain_core.runnables import RunnablePassthrough

hyde_chain = RunnablePassthrough.assign(hypothetical_document=qa_no_context)

hyde_chain.invoke(
    {
        "question": "how to use multi-modal models in a chain and turn chain into a rest api"
    }
)

### 后续可能提分点
- 引入LLM
   * LLM 递归判断/抽取
   * rag-fusion 查询改写
   * 构建知识图谱



### 注意：
- 在分块、重排等过程中可以使用公开库和模型，但禁止使用LLM直接生成最终答案。
- 禁止使用LLM继续调整精排得到的文本块，如压缩文本块长度；
- 禁止使用LLM直接从文档获取问题答案。